# Calculate the mean value of the USD-EUR

In [ ]:
import pandas as pd
import numpy as np
import json
import datetime
import pickle
import functions as fn
import matplotlib.pyplot as plt
%matplotlib inline
import requests

## Select wanted exchange rate

In [ ]:
exchange_currency = input()

In [ ]:
desired_currency = input()

## Import data to calculate the exchange rate

In [ ]:
#Retrieving my api keys information to access the Alpha Advantage API.
keys = fn.get_keys("/Users/jjherranzsarrion/.secret/alphaadvantage.json")
api_key = keys['api_key']

In [ ]:
# Get the fx rate from Alpha Advantage 
response_json = fn.get_fx_rates(url, exchange_currency, desired_currency)

In [ ]:
# Extract only the adjusted closing price
response_df = fn.get_adjusted_rate(response_json)

## Graphing the exchange rate

In [ ]:
# Calculate 30 Day Moving Average, Std Deviation, Upper Band and Lower Band
response_bb_df = fn.get_bollinger_bands(response_df)

In [ ]:
# Retrieve todays date
today = date.today()

In [ ]:
# Return graphic showing the previous monthly history of the exchange rate in question and return recommendation
response_graphic = fn.get_graphical_view(response_bb_df, exchange_currency, desired_currency, today)

There is a problem with the dates on the x axis so check that out